In [1]:
import os
import numpy as np
import torch
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import torch 
import time
from audioldm import build_model
from audioldm.latent_diffusion.ddim import DDIMSampler
from audioldm.pipeline import duration_to_latent_t_size

sr = 16000

#test
#Saturn is working
#Saturn is working
#Saturn is working

In [2]:

model = build_model(model_name="audioldm-l-full")

torch.cuda.reset_peak_memory_stats()
torch.cuda.synchronize()

modelmem = torch.cuda.memory_allocated() / (1024 * 1024) 

print(f"{modelmem} MiB")
print(torch.cuda.memory_summary())

Load AudioLDM: %s audioldm-l-full
DiffusionWrapper has 739.14 M params.
DiffusionWrapper has 739.14 M params.


/home/manish/Ayush/audioldm/ldmenv/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/manish/Ayush/audioldm/ldmenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/manish/Ayush/audioldm/ldmenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/manish/Ayush/audioldm/ldmenv/lib/python3.8/site-packages/torch/functional.py:513: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internal

6706.32861328125 MiB
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   6706 MiB |   6706 MiB |  13410 MiB |   6703 MiB |
|       from large pool |   6547 MiB |   6547 MiB |  13093 MiB |   6545 MiB |
|       from small pool |    158 MiB |    158 MiB |    317 MiB |    158 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   6706 MiB |   6706 MiB |  13410 MiB |   6703 MiB |
|       from large pool |   6547 MiB |   65

In [ ]:
prompt = "90s rock song with electric guitar and heavy drums."
duration = 30.0  

In [4]:
model.cond_stage_model.embed_mode = "text"
text_list = [prompt] 

text_emb = model.get_learned_conditioning(text_list)

unconditional_text = [""] 
unconditional_embedding = model.get_learned_conditioning(unconditional_text)

In [5]:
sampler = DDIMSampler(model)

torch.cuda.synchronize()
latent_size = duration_to_latent_t_size(duration)
shape = [model.channels, latent_size, model.latent_f_size]
z = torch.randn([1] + shape).to(model.device)

ddim_steps = 200
ddim_eta = 0.1
unconditional_guidance_scale = 3.0

memory_before_diffusion = torch.cuda.memory_allocated() / (1024 * 1024)
torch.cuda.reset_peak_memory_stats()

# Run diffusion
start_time = time.time()
with torch.no_grad(): 
    samples, _ = sampler.sample(
        S=ddim_steps,  
        batch_size=1,
        shape=shape,
        conditioning=text_emb,
        unconditional_conditioning=unconditional_embedding,
        unconditional_guidance_scale=unconditional_guidance_scale,
        eta=ddim_eta,
        verbose=False,
        x_T=z
    )
torch.cuda.synchronize()

# Get time and memory
diffusion_time = time.time() - start_time
isolated_peak = torch.cuda.max_memory_allocated() / (1024 * 1024)  # Convert to MiB
current_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MiB
isolated_cost = isolated_peak - memory_before_diffusion

print(f"DIFFUSION ISOLATED PEAK: {isolated_peak:.1f} MiB (cost: +{isolated_cost:.1f} MiB)")
print(f"DIFFUSION CURRENT MEMORY: {current_memory:.1f} MiB")
print(f"DIFFUSION TIME: {diffusion_time:.1f}s ({diffusion_time/ddim_steps:.3f}s per step)")
print(torch.cuda.memory_summary())

DDIM Sampler:   5%|▌         | 10/200 [00:00<00:08, 21.23it/s]

DDIM Sampler: 100%|██████████| 200/200 [00:08<00:00, 23.74it/s]

DIFFUSION ISOLATED PEAK: 6853.7 MiB (cost: +139.1 MiB)
DIFFUSION CURRENT MEMORY: 6714.9 MiB
DIFFUSION TIME: 8.4s (0.042s per step)
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   6714 MiB |   6853 MiB | 757689 MiB | 750974 MiB |
|       from large pool |   6555 MiB |   6690 MiB | 688085 MiB | 681529 MiB |
|       from small pool |    159 MiB |    169 MiB |  69603 MiB |  69444 MiB |
|---------------------------------------------------------------------------|
| Active me

In [6]:
# VAE DECODER VRAM MEASUREMENT
# Get memory before VAE and reset peak tracker
memory_before_vae = torch.cuda.memory_allocated() / (1024 * 1024)
torch.cuda.reset_peak_memory_stats()

# Run VAE
start_time = time.time()
with torch.no_grad(): 
    mel_spectrogram = model.decode_first_stage(samples)
torch.cuda.synchronize()
vae_time = time.time() - start_time

# Get memory usage
isolated_peak = torch.cuda.max_memory_allocated() / (1024 * 1024)  # Convert to MiB
current_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MiB
isolated_cost = isolated_peak - memory_before_vae

print(f"VAE DECODER ISOLATED PEAK: {isolated_peak:.1f} MiB (cost: +{isolated_cost:.1f} MiB)")
print(f"VAE DECODER CURRENT MEMORY: {current_memory:.1f} MiB")
print(f"VAE TIME: {vae_time:.1f}s")
print(torch.cuda.memory_summary())

VAE DECODER ISOLATED PEAK: 6845.1 MiB (cost: +130.1 MiB)
VAE DECODER CURRENT MEMORY: 6715.1 MiB
VAE TIME: 0.0s
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   6715 MiB |   6845 MiB | 759311 MiB | 752596 MiB |
|       from large pool |   6555 MiB |   6685 MiB | 689706 MiB | 683151 MiB |
|       from small pool |    159 MiB |    159 MiB |  69604 MiB |  69445 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   671

In [7]:
memory_before_vocoder = torch.cuda.memory_allocated() / (1024 * 1024)
torch.cuda.reset_peak_memory_stats()

start_time = time.time()
with torch.no_grad(): 
    waveform = model.mel_spectrogram_to_waveform(mel_spectrogram)[0]
torch.cuda.synchronize()
vocoder_time = time.time() - start_time

# Get memory usage
isolated_peak = torch.cuda.max_memory_allocated() / (1024 * 1024)  # Convert to MiB
current_memory = torch.cuda.memory_allocated() / (1024 * 1024)  # Convert to MiB
isolated_cost = isolated_peak - memory_before_vocoder

print(f"VOCODER ISOLATED PEAK: {isolated_peak:.1f} MiB (cost: +{isolated_cost:.1f} MiB)")
print(f"VOCODER CURRENT MEMORY: {current_memory:.1f} MiB")
print(f"VOCODER TIME: {vocoder_time:.1f}s")

ipd.display(ipd.Audio(waveform, rate=sr))
print(torch.cuda.memory_summary())

VOCODER ISOLATED PEAK: 6777.8 MiB (cost: +62.8 MiB)
VOCODER CURRENT MEMORY: 6715.1 MiB
VOCODER TIME: 0.1s


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   6715 MiB |   6777 MiB | 762392 MiB | 755677 MiB |
|       from large pool |   6555 MiB |   6618 MiB | 692778 MiB | 686222 MiB |
|       from small pool |    159 MiB |    159 MiB |  69614 MiB |  69455 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   6715 MiB |   6777 MiB | 762392 MiB | 755677 MiB |
|       from large pool |   6555 MiB |   6618 MiB | 692778 MiB |